In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../../data/processed/cleaned/parse_legal_cases.csv")

/var/folders/jv/qp9hncvx77d6nxpff8g0q_n80000gn/T/ipykernel_15549/3953116954.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../data/processed/cleaned/parse_legal_cases.csv")


In [3]:


pd.set_option('display.max_colwidth', None)
print(df['opinion_text'].head(10))


0                       FOR PUBLICATION\n\n   UNITED STATES COURT OF APPEALS\n        FOR THE NINTH CIRCUIT\n\nMURPHY COMPANY, an Oregon\n                                            No. 19-35921\ncorporation; MURPHY TIMBER\nINVESTMENTS, LLC, an Oregon\nlimited liability company,                    D.C. No.\n                 Plaintiffs-Appellants,   1:17-cv-00285-CL\n\n  v.\n                                             OPINION\nJOSEPH R. BIDEN, JR., in his official\ncapacity as President of the United\nStates of America; DEBRA A.\nHAALAND, in her official capacity as\nSecretary of Interior; U.S.\nDEPARTMENT OF THE INTERIOR,\n               Defendants-Appellees,\n\nSODA MOUNTAIN WILDERNESS\nCOUNCIL; KLAMATH-SISKIYOU\nWILDLANDS CENTER; OREGON\nWILD; WILDERNESS SOCIETY,\n           Intervenor-Defendants-\n           Appellees.\n\n       Appeal from the United States District Court\n                 for the District of Oregon\n       Michael J. McShane, District Judge, Presiding\n2        

In [7]:

import re

def extract_all_disposition_lines(text, min_length=40):
    if not isinstance(text, str):
        return []

    sentences = re.split(r'(?<=[\.\n])\s+', text)
    
    result = []
    buffer = ''
    for sentence in sentences:
        buffer += ' ' + sentence.strip()
        if len(buffer.strip()) >= min_length:
            if re.search(r'\b(affirm|reverse|remand|dismiss|vacate|deny|grant|conclude|find|hold)\b', buffer, re.IGNORECASE):
                result.append(buffer.strip())
            buffer = ''  # Завжди скидаємо після перевірки

    return result


# Створимо нову колонку з витягнутими рішеннями
df['opinion_text_clean'] = df['opinion_text'].str.replace('\n', ' ', regex=False).str.replace(r'\s+', ' ', regex=True).str.strip()


# 🔹 Витягнути речення з рішеннями
df['decision_lines'] = df['opinion_text_clean'].apply(lambda x: extract_all_disposition_lines(x, min_length=50))

# 🔹 Об'єднати в один рядок (за бажанням)
df['decision_lines_joined'] = df['decision_lines'].apply(lambda lines: ' '.join(lines) if lines else None)




In [9]:
# Перевіримо перші 10
print(df['decision_lines'].head(20))

0     [MURPHY CO. V. BIDEN 3 interest. The Oregon and California Railroad and Coos Bay Wagon Road Grant Lands Act (“O&C Act”) addresses the use of timberlands in the southwest corner of Oregon., 6 MURPHY CO. V. BIDEN OPINION McKEOWN, Circuit Judge: This case calls on us to consider the intersection of the Antiquities Act, adopted in 1906, and the Oregon and California Railroad and Coos Bay Wagon Road Grant Lands Act (“O&C Act”), adopted in 1937., Because we conclude that we have jurisdiction to hear Murphy’s challenge, we next evaluate whether the Proclamation’s restriction on logging is consistent with the O&C Act., We affirm the district 8 MURPHY CO. V. BIDEN court’s grant of summary judgment in favor of the United States and Soda Mountain., In 1866, the United States made a grant of purportedly “public lands” to private railroad companies to facilitate the construction of a rail line between Oregon and California., 10, 1869, ch. 27, 16 Stat. 47. But the railroads violated the terms 